In [1]:
from gbfs.services import SystemDiscoveryService
import pandas as pd
import numpy as np

In [2]:
ds = SystemDiscoveryService()

In [3]:
systems = pd.DataFrame(ds.systems)
systems[systems["Country Code"]=="US"].Location.value_counts().sort_index().reset_index().values.tolist()

# ['Seattle, WA', 4]
# ['Los Angeles, CA', 6],

[['Akron, OH', 1],
 ['Albuquerque, NM', 1],
 ['Alexandria, VA', 3],
 ['Ann Arbor, MI', 1],
 ['Arlington, VA', 3],
 ['Asbury Park, NJ', 1],
 ['Asbury Park, NY', 1],
 ['Aspen, CO', 1],
 ['Atlanta, GA', 3],
 ['Austin, TX', 3],
 ['Aventura, FL', 1],
 ['Baltimore, MD', 3],
 ['Blackburg, VA', 1],
 ['Boise, ID', 1],
 ['Boston, MA', 1],
 ['Boulder, CO', 1],
 ['Brookline, MA', 1],
 ['Buffalo, NY', 1],
 ['Charlotte, NC', 1],
 ['Charlotte,NC', 1],
 ['Chattanooga, TN', 1],
 ['Chicago, IL', 6],
 ['Cincinnati, OH', 1],
 ['Clarksville, TN', 1],
 ['Clemson, SC', 1],
 ['Cleveland, OH', 5],
 ['Colorado Springs, CO, US', 1],
 ['Columbus, OH', 5],
 ['Coral Gables, FL', 1],
 ['Culver City, CA', 1],
 ['Dallas,TX', 1],
 ['Dayton, OH', 1],
 ['Denver, CO', 2],
 ['Des Moines, IA', 1],
 ['Detroit, MI', 6],
 ['Durham NS', 1],
 ['Durham, NC', 2],
 ['East Lansing, MI', 1],
 ['El Paso, TX', 1],
 ['Encinitias, CA', 1],
 ['Eugene, OR', 1],
 ['Fairfax, VA', 2],
 ['Fayetteville, AR', 1],
 ['Fort Collins, CO', 1],
 ['For

In [4]:
[x.get('System ID') for x in ds.systems if 'San Diego' in x.get('Location')]

['lyft_san', 'Link_San_Diego', 'spin san_diego', 'spin uc_san_diego']

In [5]:
[x.get('System ID') for x in ds.systems if 'Los Angeles' in x.get('Location')]

['bird-los-angeles',
 'Link_Los_Angeles',
 '10',
 'lyft_lax',
 'bcycle_lametro',
 'spin los_angeles']

In [6]:
[x.get('System ID') for x in ds.systems if 'Seattle' in x.get('Location')]

['lime_seattle', 'Link_Seattle', '33', 'spin seattle']

In [7]:
import time
from geopy.distance import great_circle
from tqdm import tqdm
import pickle
import datetime

In [8]:
clients = {}

for x in ds.systems:
    if 'Seattle' in x.get('Location') or 'Los Angeles' in x.get('Location'):
        clients[x["System ID"]] = ds.instantiate_client(x["System ID"])

In [9]:
clients

{'bird-los-angeles': GBFSClient('https://mds.bird.co/gbfs/v2/public/los-angeles/gbfs.json', 'en'),
 'lime_seattle': GBFSClient('https://data.lime.bike/api/partners/v2/gbfs/seattle/gbfs.json', 'en'),
 'Link_Los_Angeles': GBFSClient('https://mds.linkyour.city/gbfs/us_ca_los_angeles/gbfs.json', 'en'),
 'Link_Seattle': GBFSClient('https://mds.linkyour.city/gbfs/us_wa_seattle/gbfs.json', 'en'),
 '10': GBFSClient('https://gbfs.hopr.city/api/gbfs/10', 'en'),
 'lyft_lax': GBFSClient('https://s3.amazonaws.com/lyft-lastmile-production-iad/lbs/lax/gbfs.json', 'en'),
 'bcycle_lametro': GBFSClient('https://gbfs.bcycle.com/bcycle_lametro/gbfs.json', 'en'),
 '33': GBFSClient('https://gbfs.hopr.city/api/gbfs/33/', 'en'),
 'spin los_angeles': GBFSClient('https://gbfs.spin.pm/api/gbfs/v2_3/los_angeles/gbfs', 'en'),
 'spin seattle': GBFSClient('https://gbfs.spin.pm/api/gbfs/v2_3/seattle/gbfs', 'en')}

In [ ]:
bikes = {client:{} for client in clients}

i = 1
while(True):
    if i % 120 == 0:
        # daily save:
        timestamp = str(f"{datetime.datetime.now():%Y-%m-%d_%H}")
        with open(f'bikes_{timestamp}.pickle', 'wb') as f:
            pickle.dump(bikes, f)
        print(sum(len(bikes[c][f]) for c in bikes for f in bikes[c]))
        bikes = {client:{} for client in clients}
    
    for client in clients:
        try:
            result = clients[client].request_feed('free_bike_status')   
            timestamp = result["last_updated"]
            
            for bike in result["data"]["bikes"]:
                bike_id = bike["bike_id"]
                tmp = bikes[client].get(bike_id, [])
                if len(tmp) > 0:
                    last = tmp[-1]
                    if great_circle((bike["lat"],bike["lon"]), 
                                    (last[1], last[2])).meters > 100 or \
                        last[3] != bike["is_reserved"] or \
                        last[4] != bike["is_disabled"] :
                        
                        tmp.append((timestamp, 
                                    np.round(bike["lat"],5), 
                                    np.round(bike["lon"],5), 
                                    bike['is_reserved'], bike['is_disabled'],))
                        bikes[client][bike_id] = tmp
                else:
                    tmp.append((timestamp, 
                                np.round(bike["lat"]), 
                                np.round(bike["lon"]), 
                                bike['is_reserved'], bike['is_disabled'],))
                    bikes[client][bike_id] = tmp
        except Exception as e:
            pass
    time.sleep(60)
    i += 1
    
# daily save:
timestamp = str(f"{datetime.datetime.now():%Y-%m-%d_%H}")
print(sum(len(bikes[c][f]) for c in bikes for f in bikes[c]))
with open(f'bikes_{timestamp}.pickle', 'wb') as f:
    pickle.dump(bikes, f)

696586
705872
695760
702650
703075
689364
694565
693018
698519
693901
709845
711872
705323
710131
716958
718666
716954
734248
740264
733517
732603
739829
741258
738293
734848
738613
737781
737463
752885
752558
743465
744099
754212
753443
767992
774747
771069
765913
765228
769404


In [64]:
a = 0
for c in bikes:
    for f in bikes[c]:
        a += len(bikes[c][f])

In [65]:
a

47084

In [ ]:
!mkdir data/SD/gbfs/

In [21]:
bike["lat"]

34.102092

In [ ]:
from time import sleep

In [ ]:
for i in range(1000):
    result = client.request_feed('free_bike_status')
    df = pd.DataFrame(result["data"]["bikes"])
    df.to_csv(f'data/SD/gbfs/{result["last_updated"]}.csv')
    sleep(30)

In [ ]:
str(result["last_updated"])

In [ ]:
pd.DataFrame(result["data"]["bikes"])

In [ ]:
result

In [33]:

timestamp

datetime.datetime(2023, 4, 17, 12, 30, 11)

In [47]:
bikes

{'a176655f-598c-4dca-adfd-8d46ae51bd22': [[datetime.datetime(2023, 4, 17, 13, 59, 9),
   47.61204,
   -122.34366,
   False,
   False]],
 'e93b89f7-e24b-4913-8f95-827aa9bf3020': [[datetime.datetime(2023, 4, 17, 13, 59, 9),
   47.58088,
   -122.32733,
   False,
   False]],
 '18ae492c-2ba4-49ed-9006-ece279511732': [[datetime.datetime(2023, 4, 17, 13, 59, 9),
   47.61627,
   -122.32665,
   False,
   True]],
 '5f9abc2b-25e0-4a2f-8e7f-c1baebf8ff19': [[datetime.datetime(2023, 4, 17, 13, 59, 9),
   47.66237,
   -122.39095,
   False,
   True]],
 '1759b3c3-8b98-46d6-8369-f37e31d7817b': [[datetime.datetime(2023, 4, 17, 13, 59, 9),
   47.61193,
   -122.33287,
   False,
   False]],
 'a2d9f3b2-c5dc-4ddc-a56f-ea852a59b93b': [[datetime.datetime(2023, 4, 17, 13, 59, 9),
   47.58868,
   -122.33813,
   False,
   False]],
 '821fbb5f-6f44-41de-9347-5f1b5f53a5bc': [[datetime.datetime(2023, 4, 17, 13, 59, 9),
   47.54241,
   -122.28332,
   False,
   False]],
 '8aa34792-b677-4f8a-80fc-28c6b8e7fea7': [[datetim